# Task 2

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

In [12]:
import numpy as np

class Dataloader:
    def __init__(self):
        self.t = None
        self.u = None
        self.z = None

    def load(self, filename):
        quadrotor_data = np.loadtxt(filename, delimiter=',') # load data from file
        self.t = quadrotor_data[:, 0]
        self.u = quadrotor_data[:, 1:4]
        self.z = quadrotor_data[:, 4:7]
        print(f"Data loaded from {filename}")
        return self.t, self.u, self.z


In [17]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

class KalmanFilter:
    def __init__(self, t, u ,z):
        self.t = t
        self.u = u
        self.z = z
        self.xhat = None
        self.P = None

    def predict(self, xhat_minus_1, P_k_minus_1, A, Q, G, u_k_minus_1, m):
        dt = self.t[1] - self.t[0]
        F = np.eye(A.shape[0]) + A*dt
        xhat_pred = F @ xhat_minus_1 + G @ u_k_minus_1*m
        P_pred = F@P_k_minus_1@F.T + Q
        return xhat_pred, P_pred
    
    def update(self, x_pred, P_pred, z_k, H, R):
        S = H@P_pred@H.T + R
        K = P_pred @ H @ np.linalg.inv(S)
        y = z_k - H @ x_pred
        xhat_k = x_pred + K @ y
        P_k = (np.eye(K.shape[0]) - K@H) @ P_pred
        return xhat_k, P_k
    
    def run_filter(self, H, R, Q, m = 0.027, param = 'pos'):
        n = len(self.t)
        self.xhat = np.zeros((n, 6))
        self.P = np.zeros((n, 6, 6))
        A , G = self.get_system_matrices()

        if param == 'pos':
            self.xhat[0] = np.concatenate([self.z[0], np.zeros(3)])
        elif param == 'vel':
            self.xhat[0] = np.concatenate([np.zeros(3), np.zeros(3)])

        self.P[0] = np.diag([0.1, 0.1, 0.1, 0.5, 0.5, 0.5])

        # for k in range(1, n):
        #     xhat_pred, P_pred = self.predict(self.xhat[k-1], self.P[k-1], A, Q, G, self.u[k-1], m)
        #     self.xhat[k], self.P[k] = self.update(xhat_pred, P_pred, self.z[k], H, R)

        for k in range(1, n):
            xhat_pred, P_pred = self.predict(self.xhat[k-1], self.P[k-1], A, Q, G, self.u[k-1], m)
            self.xhat[k], self.P[k] = self.update(xhat_pred, P_pred, self.z[k], H, R)
            return self.xhat, self.P
        
    def get_system_matrices(self):
        dt = self.t[1] - self.t[0]
        A = np.array([[1, 0, 0, dt, 0, 0],
                      [0, 1, 0, 0, dt, 0],
                      [0, 0, 1, 0, 0, dt],
                      [0, 0, 0, 1, 0, 0],
                      [0, 0, 0, 0, 1, 0],
                      [0, 0, 0, 0, 0, 1]])
        G = np.array([[dt**2/2, 0, 0],
                      [0, dt**2/2, 0],
                      [0, 0, dt**2/2],
                      [dt, 0, 0],
                      [0, dt, 0],
                      [0, 0, dt]])
        return A, G
    
    def plot_results(self):
        if self.xhat is None:
            print("Run the filter first")
            return
        p_hat = self.xhat[:, :3]
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot(p_hat[:, 0], p_hat[:, 1], p_hat[:, 2])
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        plt.show()

    def animate_results(self):
        if self.xhat is None:
            print("Data not yet filtered. Run the filter first.")
            return

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')

        # Setting the bounds for the axes
        p_hat = self.xhat[:, :3]
        max_range = np.array([p_hat[:,0].max()-p_hat[:,0].min(), p_hat[:,1].max()-p_hat[:,1].min(), p_hat[:,2].max()-p_hat[:,2].min()]).max() / 2.0
        mid_x = (p_hat[:,0].max()+p_hat[:,0].min()) * 0.5
        mid_y = (p_hat[:,1].max()+p_hat[:,1].min()) * 0.5
        mid_z = (p_hat[:,2].max()+p_hat[:,2].min()) * 0.5
        ax.set_xlim(mid_x - max_range, mid_x + max_range)
        ax.set_ylim(mid_y - max_range, mid_y + max_range)
        ax.set_zlim(mid_z - max_range, mid_z + max_range)

        line, = ax.plot([], [], [], 'r-', linewidth=2)

        def init():
            line.set_data([], [])
            line.set_3d_properties([])
            return line,

        def animate(i):
            line.set_data(p_hat[:i, 0], p_hat[:i, 1])
            line.set_3d_properties(p_hat[:i, 2])
            return line,

        anim = FuncAnimation(fig, animate, init_func=init,
                             frames=len(self.t), interval=20, blit=True)

        plt.show()

In [18]:
# Load data
dataloader = Dataloader()
t, u, z = dataloader.load("Data/kalman_filter_data_mocap.txt")

# Initialize and run Kalman Filter
kf = KalmanFilter(t, u, z)
H = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0]])
R = np.ones((3, 3))
Q = np.diag([0.01, 0.01, 0.01, 0.001, 0.001, 0.001])
kf.run_filter(H, R, Q, 'pos')
kf.plot_results()


Data loaded from Data/kalman_filter_data_mocap.txt


UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('float64'), dtype('<U3')) -> None